In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
from preprocessing import traer_datasets
from preprocessing import separar_dataset
from preprocessing import aplicar_one_hot_encoding
from preprocessing import aplicar_ordinal_encoding
from preprocessing import feature_engineering_general

Traemos datasets bla bla bla

In [5]:
df, df_sin_target, solo_target = traer_datasets()

In [7]:
df

,barrio,dia,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,id,llovieron_hamburguesas_hoy,mm_evaporados_agua,mm_lluvia_dia,nubosidad_tarde,nubosidad_temprano,presion_atmosferica_tarde,presion_atmosferica_temprano,rafaga_viento_max_direccion,rafaga_viento_max_velocidad,temp_max,temp_min,temperatura_tarde,temperatura_temprano,velocidad_viendo_tarde,velocidad_viendo_temprano,llovieron_hamburguesas_al_dia_siguiente
43694,Agronomía,2008-12-01,Oestenoroeste,Oeste,NaN,22.0,71.0,0,no,NaN,0.6,NaN,8.0,1007.1,1007.7,Oeste,44.0,22.9,13.4,21.8,16.9,24.0,20.0,no
1607,Agronomía,2008-12-02,Oestesuroeste,Nornoreste,NaN,25.0,44.0,1,no,NaN,0.0,NaN,NaN,1007.8,1010.6,Oestenoroeste,44.0,25.1,7.4,24.3,17.2,22.0,4.0,no
100483,Agronomía,2008-12-03,Oestesuroeste,Oeste,NaN,30.0,38.0,2,no,NaN,0.0,2.0,NaN,1008.7,1007.6,Oestesuroeste,46.0,25.7,12.9,23.2,21.0,26.0,19.0,no
29574,Agronomía,2008-12-04,Este,Sureste,NaN,16.0,45.0,3,no,NaN,0.0,NaN,NaN,1012.8,1017.6,Noreste,24.0,28.0,9.2,26.5,18.1,9.0,11.0,no
42669,Agronomía,2008-12-05,Noroeste,Estenoreste,NaN,33.0,82.0,4,no,NaN,1.0,8.0,7.0,1006.0,1010.8,Oeste,41.0,32.3,17.5,29.7,17.8,20.0,7.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46782,Palermo cheto,2017-06-20,Este,Estesureste,NaN,27.0,59.0,145454,no,NaN,0.0,NaN,NaN,1021.2,1024.7,Este,31.0,21.8,3.5,20.9,9.4,13.0,15.0,no
48978,Palermo cheto,2017-06-21,Estenoreste,Sureste,NaN,24.0,51.0,145455,no,NaN,0.0,NaN,NaN,1020.3,1024.6,Este,31.0,23.4,2.8,22.4,10.1,11.0,13.0,no
21868,Palermo cheto,2017-06-22,Norte,Sureste,NaN,21.0,56.0,145456,no,NaN,0.0,NaN,NaN,1019.1,1023.5,Nornoreste,22.0,25.3,3.6,24.5,10.9,9.0,13.0,no
78715,Palermo cheto,2017-06-24,Norte,Sursureste,NaN,24.0,51.0,145458,no,NaN,0.0,2.0,3.0,1016.5,1019.4,Sureste,28.0,27.0,7.8,26.0,15.1,7.0,13.0,no
